In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
with open(r"C:\Users\Bijan Emadi\.secret\yelp_api.json") as f: 
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [3]:
LOCATION = 'Oakland, CA, 94501'
TERM = 'PHO'

In [4]:
JSON_FILE = f"Data/results_in_progress_pho.json"
JSON_FILE

'Data/results_in_progress_pho.json'

In [6]:
file_exists = os.path.isfile(JSON_FILE)
if file_exists == False:
    folder = os.path.dirname(JSON_FILE)
    if len(folder)>0:
        os.makedirs(folder, exist_ok=True)
        
    print(f"[i] {JSON_FILE} not found. Saving empty list to file.")
    
    with open(JSON_FILE, 'w') as f:
        json.dump([],f)
else:
    print(f"[i] {JSON_FILE} already exists.")
    

[i] Data/results_in_progress_pho.json already exists.


In [7]:
with open(JSON_FILE, 'r') as f:
    previous_results = json.load(f)
    
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [9]:
results = yelp_api.search_query(location=LOCATION,
                                     term=TERM,
                                     offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [10]:
total_results = results['total']
total_results

125

In [12]:
results_per_page = len(results['businesses'])
results_per_page

20

In [13]:
import time, math
n_pages = math.ceil((results['total']-n_results)/results_per_page)
n_pages

7

In [14]:
previous_results.extend(results['businesses'])
with open(JSON_FILE,'w') as f:
    json.dump(previous_results,f)

In [15]:
pip install tqdm

Note: you may need to restart the kernel to use updated packages.


In [16]:
from tqdm.notebook import tqdm_notebook
import time
for i in tqdm_notebook(range(n_pages)):
    time.sleep(0.2)

  0%|          | 0/7 [00:00<?, ?it/s]

In [18]:
for i in tqdm_notebook(range(1, n_pages+1)):
    time.sleep(0.2)
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
        
    n_results = len(previous_results)
    results = yelp_api.search_query(location=LOCATION,
                                   term=TERM,
                                   offset=n_results)
    previous_results.extend(results['businesses'])
    with open(JSON_FILE, 'w') as f:
        json.dump(previous_results, f)

  0%|          | 0/7 [00:00<?, ?it/s]

In [19]:
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,cW8_VvgXmb-d_3tWzBuQow,sidestreet-pho-alameda,Sidestreet Pho,https://s3-media3.fl.yelpcdn.com/bphoto/ABHLry...,False,https://www.yelp.com/biz/sidestreet-pho-alamed...,922,"[{'alias': 'vietnamese', 'title': 'Vietnamese'...",4.0,"{'latitude': 37.7629323, 'longitude': -122.245...","[delivery, pickup]",$$,"{'address1': '2304 Encinal Ave', 'address2': '...",+15108085298,(510) 808-5298,1310.276278
1,A-YPiGfW6sXwKaR4_HSe5w,pho-nation-express-alameda-2,Pho Nation Express,https://s3-media3.fl.yelpcdn.com/bphoto/a7lSu6...,False,https://www.yelp.com/biz/pho-nation-express-al...,15,"[{'alias': 'vietnamese', 'title': 'Vietnamese'}]",4.5,"{'latitude': 37.765615, 'longitude': -122.24203}","[delivery, pickup]",NaN,"{'address1': '2353 Santa Clara Ave', 'address2...",+15102394187,(510) 239-4187,1426.908471
2,9khVsJM_HJUlnQBm8Z9AuQ,mint-leaf-vietnamese-restaurant-alameda-2,Mint Leaf Vietnamese Restaurant,https://s3-media2.fl.yelpcdn.com/bphoto/PziXGS...,False,https://www.yelp.com/biz/mint-leaf-vietnamese-...,439,"[{'alias': 'vietnamese', 'title': 'Vietnamese'...",4.0,"{'latitude': 37.7826864, 'longitude': -122.272...","[delivery, pickup]",$$,"{'address1': '831 Marina Village Pkwy', 'addre...",+15105222758,(510) 522-2758,2123.210276
3,ouAw6oihgt3Ricwcbf1fMA,pho-anh-dao-alameda-alameda,Pho Anh Dao Alameda,https://s3-media2.fl.yelpcdn.com/bphoto/pk5SKY...,False,https://www.yelp.com/biz/pho-anh-dao-alameda-a...,143,"[{'alias': 'vietnamese', 'title': 'Vietnamese'}]",3.5,"{'latitude': 37.778843, 'longitude': -122.276938}",[delivery],$$,"{'address1': '1919 Webster St', 'address2': ''...",+15102638577,(510) 263-8577,2002.757055
4,wrJ8EiaJSDuX4IZIIwgK5w,rang-dong-restaurant-alameda-3,Rang Dong Restaurant,https://s3-media3.fl.yelpcdn.com/bphoto/zMmgXT...,False,https://www.yelp.com/biz/rang-dong-restaurant-...,393,"[{'alias': 'vietnamese', 'title': 'Vietnamese'...",4.0,"{'latitude': 37.771191, 'longitude': -122.277115}","[delivery, pickup]",$$,"{'address1': '660 Central Ave', 'address2': 'S...",+15108143938,(510) 814-3938,1731.394832


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
120,HAmjavXikrJuPZ0XP9NrSQ,take-it-easy-oakland,Take It Easy,https://s3-media1.fl.yelpcdn.com/bphoto/SeDCUl...,False,https://www.yelp.com/biz/take-it-easy-oakland?...,229,"[{'alias': 'thai', 'title': 'Thai'}]",3.5,"{'latitude': 37.805700009529, 'longitude': -12...",[delivery],$,"{'address1': '351 17th St', 'address2': '', 'a...",+15104522442,(510) 452-2442,4136.561681
121,r7fgX1npiwTuxJUgGJY-9Q,lg-supermarket-oakland-2,LG Supermarket,https://s3-media1.fl.yelpcdn.com/bphoto/YYah5Y...,False,https://www.yelp.com/biz/lg-supermarket-oaklan...,15,"[{'alias': 'grocery', 'title': 'Grocery'}]",2.5,"{'latitude': 37.80017, 'longitude': -122.27005}",[],$$,"{'address1': '325 10th St', 'address2': '', 'a...",+15102388618,(510) 238-8618,3603.422207
122,9Kgr2MVkcabB40XGmXP90A,pill-hill-deli-oakland,Pill Hill Deli,https://s3-media1.fl.yelpcdn.com/bphoto/ZmSPDa...,False,https://www.yelp.com/biz/pill-hill-deli-oaklan...,61,"[{'alias': 'delis', 'title': 'Delis'}]",4.0,"{'latitude': 37.81927, 'longitude': -122.26481}",[delivery],$,"{'address1': '400 30th St', 'address2': 'Ste 4...",+15108348819,(510) 834-8819,5585.228697
123,MCRJyzRfmz16ZpEjYFnHtQ,sun-hop-fat-1-supermarket-oakland,Sun Hop Fat 1 Supermarket,https://s3-media1.fl.yelpcdn.com/bphoto/z6THeM...,False,https://www.yelp.com/biz/sun-hop-fat-1-superma...,89,"[{'alias': 'grocery', 'title': 'Grocery'}]",2.5,"{'latitude': 37.7946815490723, 'longitude': -1...",[],$,"{'address1': '501 E 12th St', 'address2': '', ...",+15107638888,(510) 763-8888,2836.873781
124,uAHNKMsRG3A9-DXp7922DQ,smitten-ice-cream-oakland-3,Smitten Ice Cream,https://s3-media1.fl.yelpcdn.com/bphoto/weiPp3...,False,https://www.yelp.com/biz/smitten-ice-cream-oak...,838,"[{'alias': 'icecream', 'title': 'Ice Cream & F...",4.0,"{'latitude': 37.8461514110041, 'longitude': -1...","[delivery, pickup]",$$,"{'address1': '5800 College Ave', 'address2': '...",+15102773720,(510) 277-3720,8555.370636


In [20]:
final_df.duplicated(subset='id').sum()

1

In [21]:
final_df.to_csv('Data/final_results_pho.csv.gz', compression='gzip', index=False)